# OECD Manual Download Analysis

This notebook processes the manually downloaded OECD QNA data to match the paper's specification:
- **VPVOBARSA**: Volume estimates, seasonally adjusted, 2015 PPPs

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

## 1. Load the Raw OECD Data

Note: This file is large (~200MB), loading may take a moment.

In [2]:
# Load the raw OECD manual download
# Only load necessary columns to save memory
usecols = [
    'FREQ', 'ADJUSTMENT', 'REF_AREA', 'Reference area',
    'TRANSACTION', 'Transaction', 'UNIT_MEASURE', 'Unit of measure',
    'PRICE_BASE', 'Price base', 'TIME_PERIOD', 'OBS_VALUE'
]

df_raw = pd.read_csv(
    'Data/test_OECD_manual download.csv',
    usecols=usecols,
    low_memory=False
)

print(f"Loaded {len(df_raw):,} rows")
print(f"Columns: {df_raw.columns.tolist()}")

Loaded 3,674,625 rows
Columns: ['FREQ', 'ADJUSTMENT', 'REF_AREA', 'Reference area', 'TRANSACTION', 'Transaction', 'UNIT_MEASURE', 'Unit of measure', 'PRICE_BASE', 'Price base', 'TIME_PERIOD', 'OBS_VALUE']


## 2. Explore Data Dimensions

In [3]:
# Check unique values for key dimensions
print("FREQ (Frequency):")
print(df_raw['FREQ'].value_counts())

print("\nADJUSTMENT (Seasonal adjustment):")
print(df_raw['ADJUSTMENT'].value_counts())

print("\nUNIT_MEASURE:")
print(df_raw['UNIT_MEASURE'].value_counts().head(20))

print("\nPRICE_BASE:")
print(df_raw['PRICE_BASE'].value_counts())

FREQ (Frequency):
FREQ
Q    2946556
A     728069
Name: count, dtype: int64

ADJUSTMENT (Seasonal adjustment):
ADJUSTMENT
Y    2191195
N    1483430
Name: count, dtype: int64

UNIT_MEASURE:
UNIT_MEASURE
XDC           2289401
PS             420388
H              360920
IX             196868
PC             173769
USD_PPP        171444
PD              17915
JB              16164
USD_PPP_PS      15083
XDC_USD         12673
Name: count, dtype: int64

PRICE_BASE:
PRICE_BASE
V     1538654
L      914174
_Z     797472
LR     233547
DR      67017
D       63864
Q       47396
QR      12501
Name: count, dtype: int64


In [22]:
# Check TRANSACTION codes (GDP = B1GQ)
print("TRANSACTION codes:")
trans_counts = df_raw.groupby(['TRANSACTION', 'Transaction']).size().reset_index(name='count')
print(trans_counts.sort_values('count', ascending=False).head(200))

TRANSACTION codes:
   TRANSACTION                                        Transaction   count
36          P3                      Final consumption expenditure  409091
52        P51G                      Gross fixed capital formation  408803
1          B1G                                 Value added, gross  362847
27         EMP                                   Total employment  263073
64         SAL                                          Employees  261299
65        SELF                                      Self employed  258230
2         B1GQ                             Gross domestic product  218959
59          P7                      Imports of goods and services  150566
56          P6                      Exports of goods and services  150566
14          D1                          Compensation of employees  123199
15         D11                                 Wages and salaries  121178
16         D12                    Employers' social contributions  116459
50          P5     

## 3. Filter for Paper's Specification (VPVOBARSA)

Paper uses: **VPVOBARSA** = Volume estimates, seasonally adjusted, 2015 PPPs

This means:
- `FREQ == 'Q'` (Quarterly)
- `ADJUSTMENT == 'Y'` (Seasonally adjusted)
- `PRICE_BASE` should be volume/real (check available codes)
- `UNIT_MEASURE` should be USD PPP

In [5]:
# Check Price base options with their labels
print("Price base options:")
price_base_options = df_raw.groupby(['PRICE_BASE', 'Price base']).size().reset_index(name='count')
print(price_base_options)

print("\n\nUnit measure options:")
unit_options = df_raw.groupby(['UNIT_MEASURE', 'Unit of measure']).size().reset_index(name='count')
print(unit_options.head(30))

Price base options:
  PRICE_BASE                     Price base    count
0          D                       Deflator    63864
1         DR             Deflator (rebased)    67017
2          L            Chain linked volume   914174
3         LR  Chain linked volume (rebased)   233547
4          Q                Constant prices    47396
5         QR      Constant prices (rebased)    12501
6          V                 Current prices  1538654
7         _Z                 Not applicable   797472


Unit measure options:
  UNIT_MEASURE                       Unit of measure    count
0            H                                 Hours   360920
1           IX                                 Index   196868
2           JB                                  Jobs    16164
3           PC                     Percentage change   173769
4           PD                     Percentage points    17915
5           PS                               Persons   420388
6      USD_PPP             US dollars, PPP co

In [6]:
# Filter for quarterly, seasonally adjusted data
df_q_sa = df_raw[
    (df_raw['FREQ'] == 'Q') & 
    (df_raw['ADJUSTMENT'] == 'Y')
].copy()

print(f"After filtering Q + SA: {len(df_q_sa):,} rows")

# Check what price bases and units are available
print("\nAvailable PRICE_BASE in Q+SA data:")
print(df_q_sa['PRICE_BASE'].value_counts())

print("\nAvailable UNIT_MEASURE in Q+SA data:")
print(df_q_sa['UNIT_MEASURE'].value_counts().head(15))

After filtering Q + SA: 1,758,330 rows

Available PRICE_BASE in Q+SA data:
PRICE_BASE
V     678871
L     470585
_Z    294534
LR    187328
DR     53754
D      51263
Q      12177
QR      9818
Name: count, dtype: int64

Available UNIT_MEASURE in Q+SA data:
UNIT_MEASURE
XDC           987975
IX            157979
PS            154208
PC            141003
USD_PPP       137497
H             135898
PD             17236
USD_PPP_PS     12015
XDC_USD        10091
JB              4428
Name: count, dtype: int64


In [7]:
# Filter for real volume data in USD PPP
# PRICE_BASE options to try: 'V' (volume), 'LR' (chain-linked rebased), 'L' (chain-linked)
# UNIT_MEASURE: 'USD_PPP' for PPP-converted USD

# First, let's see what combinations exist for GDP (B1GQ)
gdp_combos = df_q_sa[df_q_sa['TRANSACTION'] == 'B1GQ'].groupby(
    ['PRICE_BASE', 'Price base', 'UNIT_MEASURE', 'Unit of measure']
).size().reset_index(name='count')

print("Available GDP (B1GQ) combinations in Q+SA data:")
print(gdp_combos.to_string())

Available GDP (B1GQ) combinations in Q+SA data:
   PRICE_BASE                     Price base UNIT_MEASURE            Unit of measure  count
0           D                       Deflator           IX                      Index   6405
1          DR             Deflator (rebased)           IX                      Index   9226
2           L            Chain linked volume           PC          Percentage change  24018
3           L            Chain linked volume          XDC          National currency  17884
4          LR  Chain linked volume (rebased)           IX                      Index   8962
5          LR  Chain linked volume (rebased)      USD_PPP  US dollars, PPP converted  11606
6          LR  Chain linked volume (rebased)          XDC          National currency   9962
7           Q                Constant prices          XDC          National currency    918
8          QR      Constant prices (rebased)          XDC          National currency    611
9           V                 Cu

## 4. Select the Correct Measure

Based on exploration above, select the price_base and unit_measure that matches VPVOBARSA

In [8]:
# SELECT YOUR FILTERS HERE based on what you found above
# Adjust these values after running the exploration cells

PRICE_BASE_FILTER = 'LR'  # Try 'LR' (chain-linked rebased) or 'V' (volume)
UNIT_MEASURE_FILTER = 'USD_PPP'  # PPP-converted USD

# Filter the data
df_filtered = df_q_sa[
    (df_q_sa['PRICE_BASE'] == PRICE_BASE_FILTER) &
    (df_q_sa['UNIT_MEASURE'] == UNIT_MEASURE_FILTER)
].copy()

print(f"Filtered data: {len(df_filtered):,} rows")
print(f"Countries: {df_filtered['REF_AREA'].nunique()}")
print(f"Time periods: {df_filtered['TIME_PERIOD'].nunique()}")
print(f"Transactions: {df_filtered['TRANSACTION'].nunique()}")

Filtered data: 68,846 rows
Countries: 57
Time periods: 316
Transactions: 5


## 5. Restructure to Wide Panel Format

Convert from long format to wide panel (country × quarter) with transactions as columns

In [9]:
# Pivot to wide format
# Keep only expenditure-related transactions (B1GQ=GDP, P3=Consumption, P51G=Investment, P6=Exports, P7=Imports)
expenditure_codes = ['B1GQ', 'P3', 'P51G', 'P6', 'P7']

df_exp = df_filtered[df_filtered['TRANSACTION'].isin(expenditure_codes)].copy()

print(f"Expenditure data: {len(df_exp):,} rows")
print("\nTransactions included:")
print(df_exp.groupby(['TRANSACTION', 'Transaction']).size())

# Pivot to wide
df_wide = df_exp.pivot_table(
    index=['REF_AREA', 'Reference area', 'TIME_PERIOD'],
    columns='TRANSACTION',
    values='OBS_VALUE',
    aggfunc='mean'  # In case of duplicates
).reset_index()

# Rename columns
df_wide.columns.name = None
df_wide = df_wide.rename(columns={
    'REF_AREA': 'country',
    'Reference area': 'country_name',
    'TIME_PERIOD': 'quarter',
    'B1GQ': 'gdp',
    'P3': 'consumption',
    'P51G': 'investment',
    'P6': 'exports',
    'P7': 'imports'
})

print(f"\nWide panel: {len(df_wide):,} rows × {len(df_wide.columns)} columns")
df_wide.head()

Expenditure data: 68,846 rows

Transactions included:
TRANSACTION  Transaction                  
B1GQ         Gross domestic product           11606
P3           Final consumption expenditure    22892
P51G         Gross fixed capital formation    11428
P6           Exports of goods and services    11460
P7           Imports of goods and services    11460
dtype: int64

Wide panel: 11,618 rows × 8 columns


,country,country_name,quarter,gdp,consumption,investment,exports,imports
0,ARG,Argentina,1993-Q1,644193.60,NaN,NaN,NaN,NaN
1,ARG,Argentina,1993-Q2,662572.20,NaN,NaN,NaN,NaN
2,ARG,Argentina,1993-Q3,675891.50,NaN,NaN,NaN,NaN
3,ARG,Argentina,1993-Q4,681335.50,NaN,NaN,NaN,NaN
4,ARG,Argentina,1994-Q1,691019.60,NaN,NaN,NaN,NaN


## 6. Add Year/Quarter and Calculate YoY Growth

In [10]:
# Extract year and quarter
df_wide['year'] = df_wide['quarter'].str[:4].astype(int)
df_wide['qtr'] = df_wide['quarter'].str[-1].astype(int)

# Sort by country and time
df_wide = df_wide.sort_values(['country', 'year', 'qtr']).reset_index(drop=True)

# Calculate YoY growth (quarter t vs quarter t-4)
for col in ['gdp', 'consumption', 'investment', 'exports', 'imports']:
    if col in df_wide.columns:
        df_wide[f'{col}_yoy_pct'] = df_wide.groupby('country')[col].pct_change(periods=4) * 100

print("Sample of data with YoY:")
df_wide[['country', 'quarter', 'year', 'gdp', 'gdp_yoy_pct']].head(20)

Sample of data with YoY:


,country,quarter,year,gdp,gdp_yoy_pct
0,ARG,1993-Q1,1993,644193.60,NaN
1,ARG,1993-Q2,1993,662572.20,NaN
2,ARG,1993-Q3,1993,675891.50,NaN
3,ARG,1993-Q4,1993,681335.50,NaN
4,ARG,1994-Q1,1994,691019.60,7.27
5,ARG,1994-Q2,1994,705311.70,6.45
6,ARG,1994-Q3,1994,707091.80,4.62
7,ARG,1994-Q4,1994,715866.30,5.07
8,ARG,1995-Q1,1995,704819.70,2.00
9,ARG,1995-Q2,1995,678104.80,-3.86


## 7. Compare with Previous Data and Paper Values

In [11]:
# Check GDP values for reference countries in 2015
print("GDP values for key countries (2015 Q2):")
print("="*60)

# Paper's expected values (2015 USD billions PPP)
expected = {
    'USA': 18036,
    'DEU': 3868,
    'BRA': 3192,
    'FRA': 2647,
    'GBR': 2679
}

ref_2015 = df_wide[(df_wide['year'] == 2015) & (df_wide['qtr'] == 2)][['country', 'country_name', 'gdp']]

for country, expected_bn in expected.items():
    row = ref_2015[ref_2015['country'] == country]
    if len(row) > 0:
        raw = row['gdp'].values[0]
        # Check different scale assumptions
        print(f"{country}:")
        print(f"  Raw value: {raw:,.0f}")
        print(f"  As billions (÷1000): {raw/1000:,.1f}")
        print(f"  Expected: {expected_bn:,} bn")
        print()

GDP values for key countries (2015 Q2):
USA:
  Raw value: 19,789,136
  As billions (÷1000): 19,789.1
  Expected: 18,036 bn

DEU:
  Raw value: 4,728,510
  As billions (÷1000): 4,728.5
  Expected: 3,868 bn

BRA:
  Raw value: 3,456,948
  As billions (÷1000): 3,456.9
  Expected: 3,192 bn

FRA:
  Raw value: 3,381,079
  As billions (÷1000): 3,381.1
  Expected: 2,647 bn

GBR:
  Raw value: 3,330,880
  As billions (÷1000): 3,330.9
  Expected: 2,679 bn



## 8. Save Restructured Data

In [12]:
# Save the restructured panel
output_file = 'Data/oecd_manual_restructured.csv'
df_wide.to_csv(output_file, index=False)
print(f"Saved restructured data to: {output_file}")
print(f"Shape: {df_wide.shape}")
print(f"\nColumns: {df_wide.columns.tolist()}")

Saved restructured data to: Data/oecd_manual_restructured.csv
Shape: (11618, 15)

Columns: ['country', 'country_name', 'quarter', 'gdp', 'consumption', 'investment', 'exports', 'imports', 'year', 'qtr', 'gdp_yoy_pct', 'consumption_yoy_pct', 'investment_yoy_pct', 'exports_yoy_pct', 'imports_yoy_pct']


In [13]:
# Check country coverage
print("Countries in data:")
country_coverage = df_wide.groupby(['country', 'country_name']).agg({
    'quarter': ['min', 'max', 'count'],
    'gdp': lambda x: x.notna().sum()
}).reset_index()
country_coverage.columns = ['country', 'country_name', 'start', 'end', 'n_quarters', 'n_gdp']
print(country_coverage.to_string())

Countries in data:
      country                                       country_name    start      end  n_quarters  n_gdp
0         ARG                                          Argentina  1993-Q1  2025-Q3         131    131
1         AUS                                          Australia  1960-Q1  2025-Q3         263    263
2         AUT                                            Austria  1960-Q1  2025-Q3         263    263
3         BEL                                            Belgium  1960-Q1  2025-Q3         263    263
4         BGR                                           Bulgaria  1995-Q1  2025-Q3         123    119
5         BRA                                             Brazil  1996-Q1  2025-Q3         119    119
6         CAN                                             Canada  1960-Q1  2025-Q3         263    259
7         CHE                                        Switzerland  1960-Q1  2025-Q3         263    263
8         CHL                                              Chil

## 9. Replicate Table 1: Summary Statistics by Period

Replicate the paper's Table 1 comparing Winners vs Non-winners across time periods.

In [14]:
# Load population data
pop_df = pd.read_csv('Data/oecd_population_annual_1960_2024.csv')
print(f"Population data: {len(pop_df):,} rows")
print(pop_df.head())

# Rename country_code to country for merging
pop_df = pop_df.rename(columns={'country_code': 'country'})

# Merge population with GDP panel (need to match year)
# First, get annual population per country-year
pop_annual = pop_df.groupby(['country', 'year'])['population'].mean().reset_index()

# Merge with wide panel
df_panel = df_wide.merge(pop_annual, on=['country', 'year'], how='left')
print(f"\nPanel with population: {len(df_panel):,} rows")
print(f"Population coverage: {df_panel['population'].notna().sum():,} / {len(df_panel):,}")

Population data: 3,640 rows
  country_code country_name  year  population_million       population  \
0          AUT      Austria  1960          7047539.00 7047539000000.00   
1          AUT      Austria  1961          7086299.00 7086299000000.00   
2          AUT      Austria  1962          7129864.00 7129864000000.00   
3          AUT      Austria  1963          7175811.00 7175811000000.00   
4          AUT      Austria  1964          7223801.00 7223801000000.00   

  unit_measure                                   source  
0           PS  OECD.ELS.SAE:DSD_POPULATION@DF_POP_HIST  
1           PS  OECD.ELS.SAE:DSD_POPULATION@DF_POP_HIST  
2           PS  OECD.ELS.SAE:DSD_POPULATION@DF_POP_HIST  
3           PS  OECD.ELS.SAE:DSD_POPULATION@DF_POP_HIST  
4           PS  OECD.ELS.SAE:DSD_POPULATION@DF_POP_HIST  

Panel with population: 11,618 rows
Population coverage: 9,870 / 11,618


In [15]:
# Load World Cup winner data from the paper replication sample (has quarter-level WC data)
wc_df = pd.read_csv('Data/paper_replication_event_study_sample.csv')
print("World Cup data columns:", [c for c in wc_df.columns if 'rank' in c or 'host' in c or c in ['country', 'quarter']])
print(f"WC data rows: {len(wc_df):,}")

# Get unique country-quarter WC indicators
wc_indicators = wc_df[['country', 'quarter', 'rank1', 'rank2', 'rank3', 'rank4', 'host']].drop_duplicates()
print(f"Unique country-quarter WC records: {len(wc_indicators):,}")

# Merge WC data with panel
df_panel = df_panel.merge(
    wc_indicators,
    on=['country', 'quarter'],
    how='left'
)

# Fill NaN with 0 for rank/host columns
for col in ['rank1', 'rank2', 'rank3', 'rank4', 'host']:
    if col in df_panel.columns:
        df_panel[col] = df_panel[col].fillna(0).astype(int)

print(f"\nPanel with WC data: {len(df_panel):,} rows")
print(f"Winners (rank1=1): {df_panel['rank1'].sum()}")

World Cup data columns: ['country', 'quarter', 'host', 'rank1', 'rank2', 'rank3', 'rank4', 'host_won', 'host_top4']
WC data rows: 8,737
Unique country-quarter WC records: 8,737

Panel with WC data: 11,618 rows
Winners (rank1=1): 10


In [16]:
# Apply paper's filtering criteria (from create_paper_replication_es_csv.R)

# Paper country set (Table A2)
controls = [
    "ARG","AUS","AUT","BEL","BGR","CAN","CHL","COL","CRI","HRV","CZE","DNK","EST","FIN",
    "GRC","HUN","IND","IDN","ISL","IRL","ISR","LVA","LTU","LUX","NLD","NZL","NOR","POL",
    "PRT","ROU","SAU","SVK","SVN","SWE","CHE","TUR"
]

hosts = [
    "BRA","GBR","FRA","DEU","ITA","JPN","MEX","ZAF","KOR","ESP","RUS","USA"
]

paper_countries = sorted(set(controls + hosts))
print(f"Paper countries: {len(paper_countries)}")

# Filter to paper countries and time window
print(f"\nBefore filtering: {len(df_panel):,} rows")

df_panel = df_panel[df_panel['country'].isin(paper_countries)].copy()
print(f"After country filter: {len(df_panel):,} rows")

# Trim to paper window: 1961-Q1 to 2021-Q4
df_panel = df_panel[(df_panel['quarter'] >= '1961-Q1') & (df_panel['quarter'] <= '2021-Q4')].copy()
print(f"After time window filter (1961-Q1 to 2021-Q4): {len(df_panel):,} rows")

# Brazil special case: starts at 1998-Q2
brazil_before = len(df_panel[df_panel['country'] == 'BRA'])
df_panel = df_panel[~((df_panel['country'] == 'BRA') & (df_panel['quarter'] < '1998-Q2'))].copy()
brazil_after = len(df_panel[df_panel['country'] == 'BRA'])
print(f"Brazil special case: removed {brazil_before - brazil_after} rows (BRA now starts at 1998-Q2)")

# Filter out rows where YoY GDP growth is NA (need 4 quarters of history)
before_yoy = len(df_panel)
df_panel = df_panel[df_panel['gdp_yoy_pct'].notna()].copy()
print(f"After YoY filter: {len(df_panel):,} rows (removed {before_yoy - len(df_panel):,} with NA YoY)")

# Final check
print(f"\nFinal panel: {len(df_panel):,} rows, {df_panel['country'].nunique()} countries")
print(f"Quarter range: {df_panel['quarter'].min()} to {df_panel['quarter'].max()}")

# Check which paper countries are missing
missing = set(paper_countries) - set(df_panel['country'].unique())
if missing:
    print(f"\nCountries expected but not in data: {sorted(missing)}")

Paper countries: 48

Before filtering: 11,618 rows
After country filter: 9,724 rows
After time window filter (1961-Q1 to 2021-Q4): 8,838 rows
Brazil special case: removed 9 rows (BRA now starts at 1998-Q2)
After YoY filter: 8,737 rows (removed 92 with NA YoY)

Final panel: 8,737 rows, 48 countries
Quarter range: 1961-Q1 to 2021-Q4


In [17]:
# Assign periods and winner group (country-level: ever won)
def assign_period(year):
    if 1960 <= year <= 1980:
        return '1960–80'
    elif 1980 < year <= 2000:
        return '1980–2000'
    elif 2000 < year <= 2020:
        return '2000–20'
    return None

df_panel['period'] = df_panel['year'].apply(assign_period)

# Winner status at country level (ever had rank1 == 1)
winner_countries = df_panel[df_panel['rank1'] == 1]['country'].unique().tolist()
df_panel['is_winner_country'] = df_panel['country'].isin(winner_countries)
df_panel['winner_group'] = df_panel['is_winner_country'].map({True: 'Winner', False: 'Non-winner'})

print("Winner countries:", winner_countries)
print(f"\nWinner observations: {(df_panel['winner_group'] == 'Winner').sum():,}")
print(f"Non-winner observations: {(df_panel['winner_group'] == 'Non-winner').sum():,}")

Winner countries: ['BRA', 'DEU', 'ESP', 'FRA', 'GBR', 'ITA']

Winner observations: 1,315
Non-winner observations: 7,422


In [18]:
# Create transformed variables (matching paper units)
# GDP is in millions -> divide by 1000 for "thousands of millions" = billions
df_panel['gdp_tbl'] = df_panel['gdp'] / 1000  # thousands of (USD millions)
df_panel['pop_m'] = df_panel['population'] / 1e6  # millions
df_panel['gdp_pc'] = df_panel['gdp'] * 1e6 / df_panel['population']  # USD per capita
df_panel['gdp_yoy'] = df_panel['gdp_yoy_pct']  # YoY percent

print("Sample of transformed variables:")
df_panel[['country', 'quarter', 'gdp', 'gdp_tbl', 'pop_m', 'gdp_pc', 'gdp_yoy']].head(10)

Sample of transformed variables:


,country,quarter,gdp,gdp_tbl,pop_m,gdp_pc,gdp_yoy
4,ARG,1994-Q1,691019.60,691.02,34613491.00,0.02,7.27
5,ARG,1994-Q2,705311.70,705.31,34613491.00,0.02,6.45
6,ARG,1994-Q3,707091.80,707.09,34613491.00,0.02,4.62
7,ARG,1994-Q4,715866.30,715.87,34613491.00,0.02,5.07
8,ARG,1995-Q1,704819.70,704.82,35070020.00,0.02,2.00
9,ARG,1995-Q2,678104.80,678.10,35070020.00,0.02,-3.86
10,ARG,1995-Q3,675485.30,675.49,35070020.00,0.02,-4.47
11,ARG,1995-Q4,680664.90,680.66,35070020.00,0.02,-4.92
12,ARG,1996-Q1,699380.80,699.38,35513793.00,0.02,-0.77
13,ARG,1996-Q2,714968.40,714.97,35513793.00,0.02,5.44


In [19]:
from scipy import stats

# Helper functions for Table 1
def stars(p):
    """Return significance stars based on p-value."""
    if pd.isna(p):
        return ''
    elif p < 0.01:
        return '***'
    elif p < 0.05:
        return '**'
    elif p < 0.10:
        return '*'
    return ''

def fmt_mean_sd(mean, sd, digits=2, big=False):
    """Format mean (SD) string."""
    if pd.isna(mean) or pd.isna(sd):
        return "NA"
    if big:
        return f"{mean:,.{digits}f} ({sd:,.{digits}f})"
    return f"{mean:.{digits}f} ({sd:.{digits}f})"

def summ_row(data, var, label, digits=2, big=False):
    """Create a summary row for Winner vs Non-winner comparison."""
    d = data.dropna(subset=[var])
    
    w = d[d['winner_group'] == 'Winner'][var]
    n = d[d['winner_group'] == 'Non-winner'][var]
    
    w_mean, w_sd = w.mean(), w.std()
    n_mean, n_sd = n.mean(), n.std()
    
    # t-test
    try:
        tstat, pval = stats.ttest_ind(w, n, nan_policy='omit')
    except:
        tstat, pval = np.nan, np.nan
    
    t_str = f"{tstat:.2f}{stars(pval)}" if not pd.isna(tstat) else ""
    
    return {
        'Row': label,
        'Winner': fmt_mean_sd(w_mean, w_sd, digits, big),
        'Non-winner': fmt_mean_sd(n_mean, n_sd, digits, big),
        't-test': t_str
    }

def make_period_block(data, period_name):
    """Create a block of rows for a given period."""
    if period_name == 'Full sample':
        d = data
    else:
        d = data[data['period'] == period_name]
    
    rows = [
        summ_row(d, 'gdp_tbl', 'GDP (in thousands of 2015 US dollar millions)', digits=2, big=False),
        summ_row(d, 'pop_m', 'Population (in millions)', digits=2, big=False),
        summ_row(d, 'gdp_pc', 'GDP per capita', digits=2, big=True),
        summ_row(d, 'gdp_yoy', 'Year-on-Year GDP growth', digits=2, big=False),
    ]
    
    block = pd.DataFrame(rows)
    block['Period'] = period_name
    
    # Add counts for Full sample
    if period_name == 'Full sample':
        n_cty_w = d[d['winner_group'] == 'Winner']['country'].nunique()
        n_cty_n = d[d['winner_group'] == 'Non-winner']['country'].nunique()
        n_obs_w = len(d[d['winner_group'] == 'Winner'])
        n_obs_n = len(d[d['winner_group'] == 'Non-winner'])
        
        extra_rows = pd.DataFrame([
            {'Period': period_name, 'Row': 'Number of countries', 
             'Winner': str(n_cty_w), 'Non-winner': str(n_cty_n), 't-test': ''},
            {'Period': period_name, 'Row': 'Number of observations', 
             'Winner': str(n_obs_w), 'Non-winner': str(n_obs_n), 't-test': ''}
        ])
        block = pd.concat([block, extra_rows], ignore_index=True)
    
    return block[['Period', 'Row', 'Winner', 'Non-winner', 't-test']]

print("Helper functions defined.")

Helper functions defined.


In [20]:
# Build Table 1
period_levels = ['1960–80', '1980–2000', '2000–20', 'Full sample']

table1_manual = pd.concat([make_period_block(df_panel, p) for p in period_levels], ignore_index=True)

print("TABLE 1: Summary Statistics (Manual OECD Download)")
print("=" * 80)
display(table1_manual)

TABLE 1: Summary Statistics (Manual OECD Download)


,Period,Row,Winner,Non-winner,t-test
0,1960–80,GDP (in thousands of 2015 US dollar millions),1292.76 (512.37),536.90 (1236.35),11.99***
1,1960–80,Population (in millions),54238869.56 (13927941.80),26029197.40 (45838863.46),12.18***
2,1960–80,GDP per capita,0.02 (0.01),0.02 (0.01),1.81*
3,1960–80,Year-on-Year GDP growth,4.21 (3.00),4.73 (3.56),-2.69***
4,1980–2000,GDP (in thousands of 2015 US dollar millions),2197.33 (736.26),949.40 (2131.02),11.74***
5,1980–2000,Population (in millions),61146606.50 (22365280.55),41270295.64 (100622741.72),3.99***
6,1980–2000,GDP per capita,0.04 (0.01),0.03 (0.02),5.92***
7,1980–2000,Year-on-Year GDP growth,2.42 (1.79),3.27 (3.63),-4.63***
8,2000–20,GDP (in thousands of 2015 US dollar millions),3109.93 (819.26),1367.87 (3020.29),12.57***
9,2000–20,Population (in millions),84887559.55 (50686881.52),66509923.42 (197174508.65),2.03**


## 10. Compare with Paper's Table 1 Values

In [21]:
# Paper's Table 1 values (Full sample)
paper_values = {
    'GDP Winner': '1,908.88 (843.57)',
    'GDP Non-winner': '958.65 (2,102.79)',
    'Population Winner': '67.19 (35.15)',
    'Population Non-winner': '49.09 (145.09)',
    'GDP pc Winner': '29,258.50 (10,061.40)',
    'GDP pc Non-winner': '28,887.99 (16,921.56)',
    'YoY GDP Winner': '2.33 (3.24)',
    'YoY GDP Non-winner': '3.22 (3.78)',
    'Countries Winner': '6',
    'Countries Non-winner': '42',
    'Observations Winner': '1,295',
    'Observations Non-winner': '7,342'
}

print("COMPARISON: Your Data vs Paper's Table 1 (Full Sample)")
print("=" * 70)
print(f"{'Metric':<30} {'Paper':<25} {'Your Data':<25}")
print("-" * 70)

# Get your values from the table
full_sample = table1_manual[table1_manual['Period'] == 'Full sample']

for _, row in full_sample.iterrows():
    metric = row['Row']
    your_winner = row['Winner']
    your_nonwinner = row['Non-winner']
    
    if 'GDP' in metric and 'per capita' not in metric and 'growth' not in metric:
        print(f"GDP Winner:                    {paper_values['GDP Winner']:<25} {your_winner:<25}")
        print(f"GDP Non-winner:                {paper_values['GDP Non-winner']:<25} {your_nonwinner:<25}")
    elif 'Population' in metric:
        print(f"Population Winner:             {paper_values['Population Winner']:<25} {your_winner:<25}")
        print(f"Population Non-winner:         {paper_values['Population Non-winner']:<25} {your_nonwinner:<25}")
    elif 'per capita' in metric:
        print(f"GDP pc Winner:                 {paper_values['GDP pc Winner']:<25} {your_winner:<25}")
        print(f"GDP pc Non-winner:             {paper_values['GDP pc Non-winner']:<25} {your_nonwinner:<25}")
    elif 'growth' in metric:
        print(f"YoY GDP Winner:                {paper_values['YoY GDP Winner']:<25} {your_winner:<25}")
        print(f"YoY GDP Non-winner:            {paper_values['YoY GDP Non-winner']:<25} {your_nonwinner:<25}")
    elif 'countries' in metric:
        print(f"Countries Winner:              {paper_values['Countries Winner']:<25} {your_winner:<25}")
        print(f"Countries Non-winner:          {paper_values['Countries Non-winner']:<25} {your_nonwinner:<25}")
    elif 'observations' in metric:
        print(f"Observations Winner:           {paper_values['Observations Winner']:<25} {your_winner:<25}")
        print(f"Observations Non-winner:       {paper_values['Observations Non-winner']:<25} {your_nonwinner:<25}")

COMPARISON: Your Data vs Paper's Table 1 (Full Sample)
Metric                         Paper                     Your Data                
----------------------------------------------------------------------
GDP Winner:                    1,908.88 (843.57)         2277.67 (1040.89)        
GDP Non-winner:                958.65 (2,102.79)         1056.87 (2493.38)        
Population Winner:             67.19 (35.15)             68231372.72 (37343308.85)
Population Non-winner:         49.09 (145.09)            49649922.61 (149155948.86)
GDP pc Winner:                 29,258.50 (10,061.40)     0.04 (0.01)              
GDP pc Non-winner:             28,887.99 (16,921.56)     0.03 (0.02)              
YoY GDP Winner:                2.33 (3.24)               2.52 (3.29)              
YoY GDP Non-winner:            3.22 (3.78)               3.39 (3.97)              
Countries Winner:              6                         6                        
Countries Non-winner:          42          

## 11. Alternative: Try Chain Linked Volume (L) instead of Rebased (LR)

Let's see if using `PRICE_BASE = 'L'` produces different results.


In [24]:
# Check what's available for PRICE_BASE = 'L' (Chain linked volume, not rebased)
print("Available combinations for PRICE_BASE = 'L' in Q+SA data:")
l_combos = df_q_sa[df_q_sa['PRICE_BASE'] == 'L'].groupby(
    ['UNIT_MEASURE', 'Unit of measure']
).size().reset_index(name='count')
print(l_combos)

# Check GDP specifically
print("\n\nGDP (B1GQ) combinations for PRICE_BASE = 'L':")
gdp_l = df_q_sa[(df_q_sa['PRICE_BASE'] == 'L') & (df_q_sa['TRANSACTION'] == 'B1GQ')].groupby(
    ['UNIT_MEASURE', 'Unit of measure']
).size().reset_index(name='count')
print(gdp_l)

Available combinations for PRICE_BASE = 'L' in Q+SA data:
  UNIT_MEASURE    Unit of measure   count
0           PC  Percentage change  141003
1           PD  Percentage points   17236
2          XDC  National currency  312346


GDP (B1GQ) combinations for PRICE_BASE = 'L':
  UNIT_MEASURE    Unit of measure  count
0           PC  Percentage change  24018
1          XDC  National currency  17884


In [25]:
# Try building panel with L + XDC (national currency) - we can still compute YoY
# Note: GDP levels will be in national currency, not USD PPP

PRICE_BASE_ALT = 'L'
UNIT_MEASURE_ALT = 'XDC'  # National currency

df_alt = df_q_sa[
    (df_q_sa['PRICE_BASE'] == PRICE_BASE_ALT) &
    (df_q_sa['UNIT_MEASURE'] == UNIT_MEASURE_ALT) &
    (df_q_sa['TRANSACTION'] == 'B1GQ')  # Just GDP
].copy()

print(f"Alternative filter (L + XDC): {len(df_alt):,} rows")
print(f"Countries: {df_alt['REF_AREA'].nunique()}")

# Pivot to wide
df_alt_wide = df_alt.pivot_table(
    index=['REF_AREA', 'Reference area', 'TIME_PERIOD'],
    columns='TRANSACTION',
    values='OBS_VALUE',
    aggfunc='mean'
).reset_index()

df_alt_wide.columns.name = None
df_alt_wide = df_alt_wide.rename(columns={
    'REF_AREA': 'country',
    'Reference area': 'country_name',
    'TIME_PERIOD': 'quarter',
    'B1GQ': 'gdp'
})

# Add year and calculate YoY
df_alt_wide['year'] = df_alt_wide['quarter'].str[:4].astype(int)
df_alt_wide = df_alt_wide.sort_values(['country', 'quarter'])
df_alt_wide['gdp_yoy_pct'] = df_alt_wide.groupby('country')['gdp'].pct_change(4) * 100

print(f"\nPanel shape: {df_alt_wide.shape}")
print(df_alt_wide.head(10))

Alternative filter (L + XDC): 17,884 rows
Countries: 45

Panel shape: (6417, 6)
  country country_name  quarter      gdp  year  gdp_yoy_pct
0     AUS    Australia  1959-Q3 80842.00  1959          NaN
1     AUS    Australia  1959-Q4 81602.00  1959          NaN
2     AUS    Australia  1960-Q1 82016.00  1960          NaN
3     AUS    Australia  1960-Q2 84257.00  1960          NaN
4     AUS    Australia  1960-Q3 84438.00  1960         4.45
5     AUS    Australia  1960-Q4 84286.00  1960         3.29
6     AUS    Australia  1961-Q1 84539.00  1961         3.08
7     AUS    Australia  1961-Q2 83609.00  1961        -0.77
8     AUS    Australia  1961-Q3 83006.00  1961        -1.70
9     AUS    Australia  1961-Q4 83959.00  1961        -0.39


In [26]:
# Apply paper's filtering criteria to alternative data
controls = ["ARG","AUS","AUT","BEL","BGR","CAN","CHL","COL","CRI","HRV","CZE","DNK","EST","FIN",
            "GRC","HUN","IND","IDN","ISL","IRL","ISR","LVA","LTU","LUX","NLD","NZL","NOR","POL",
            "PRT","ROU","SAU","SVK","SVN","SWE","CHE","TUR"]
hosts = ["BRA","GBR","FRA","DEU","ITA","JPN","MEX","ZAF","KOR","ESP","RUS","USA"]
paper_countries = sorted(set(controls + hosts))

# Filter
df_alt_panel = df_alt_wide[df_alt_wide['country'].isin(paper_countries)].copy()
df_alt_panel = df_alt_panel[(df_alt_panel['quarter'] >= '1961-Q1') & (df_alt_panel['quarter'] <= '2021-Q4')]
df_alt_panel = df_alt_panel[~((df_alt_panel['country'] == 'BRA') & (df_alt_panel['quarter'] < '1998-Q2'))]
df_alt_panel = df_alt_panel[df_alt_panel['gdp_yoy_pct'].notna()]

print(f"After filtering: {len(df_alt_panel):,} rows, {df_alt_panel['country'].nunique()} countries")

# Add winner info from paper_replication_event_study_sample.csv
wc_df = pd.read_csv('Data/paper_replication_event_study_sample.csv', usecols=['country', 'quarter', 'rank1'])
wc_df = wc_df.drop_duplicates(subset=['country', 'quarter'])

df_alt_panel = df_alt_panel.merge(wc_df, on=['country', 'quarter'], how='left')
df_alt_panel['rank1'] = df_alt_panel['rank1'].fillna(0)

# Determine winner countries (ever won)
winner_countries = df_alt_panel[df_alt_panel['rank1'] == 1]['country'].unique()
df_alt_panel['winner_group'] = df_alt_panel['country'].apply(
    lambda x: 'Winner' if x in winner_countries else 'Non-winner'
)

print(f"Winner countries: {sorted(winner_countries)}")
print(f"Winners obs: {len(df_alt_panel[df_alt_panel['winner_group'] == 'Winner'])}")
print(f"Non-winners obs: {len(df_alt_panel[df_alt_panel['winner_group'] == 'Non-winner'])}")

After filtering: 5,289 rows, 43 countries
Winner countries: ['BRA', 'DEU', 'ESP', 'FRA', 'GBR', 'ITA']
Winners obs: 827
Non-winners obs: 4462


In [27]:
# Assign periods
def assign_period(year):
    if 1960 <= year <= 1980:
        return '1960–80'
    elif 1980 < year <= 2000:
        return '1980–2000'
    elif 2000 < year <= 2020:
        return '2000–20'
    return None

df_alt_panel['period'] = df_alt_panel['year'].apply(assign_period)

# Scale GDP to thousands of millions (for display - note: this is national currency, not USD PPP)
df_alt_panel['gdp_tbl'] = df_alt_panel['gdp'] / 1e6  # In millions of national currency units

# Simplified summary functions for GDP and YoY only
def summ_row_alt(data, var, label, digits=2, big=False):
    d = data.dropna(subset=[var])
    w = d[d['winner_group'] == 'Winner'][var]
    n = d[d['winner_group'] == 'Non-winner'][var]
    
    w_mean, w_sd = w.mean(), w.std()
    n_mean, n_sd = n.mean(), n.std()
    
    try:
        tstat, pval = stats.ttest_ind(w, n, nan_policy='omit')
    except:
        tstat, pval = np.nan, np.nan
    
    t_str = f"{tstat:.2f}{stars(pval)}" if not pd.isna(tstat) else ""
    
    if big:
        return {'Row': label, 'Winner': f"{w_mean:,.{digits}f} ({w_sd:,.{digits}f})",
                'Non-winner': f"{n_mean:,.{digits}f} ({n_sd:,.{digits}f})", 't-test': t_str}
    return {'Row': label, 'Winner': f"{w_mean:.{digits}f} ({w_sd:.{digits}f})",
            'Non-winner': f"{n_mean:.{digits}f} ({n_sd:.{digits}f})", 't-test': t_str}

def make_period_block_alt(data, period_name):
    if period_name == 'Full sample':
        d = data
    else:
        d = data[data['period'] == period_name]
    
    rows = [
        summ_row_alt(d, 'gdp_tbl', 'GDP (millions, national currency)', digits=2, big=True),
        summ_row_alt(d, 'gdp_yoy_pct', 'Year-on-Year GDP growth (%)', digits=2, big=False),
    ]
    
    block = pd.DataFrame(rows)
    block['Period'] = period_name
    
    if period_name == 'Full sample':
        n_cty_w = d[d['winner_group'] == 'Winner']['country'].nunique()
        n_cty_n = d[d['winner_group'] == 'Non-winner']['country'].nunique()
        n_obs_w = len(d[d['winner_group'] == 'Winner'])
        n_obs_n = len(d[d['winner_group'] == 'Non-winner'])
        
        extra = pd.DataFrame([
            {'Period': period_name, 'Row': 'Number of countries', 
             'Winner': str(n_cty_w), 'Non-winner': str(n_cty_n), 't-test': ''},
            {'Period': period_name, 'Row': 'Number of observations', 
             'Winner': str(n_obs_w), 'Non-winner': str(n_obs_n), 't-test': ''}
        ])
        block = pd.concat([block, extra], ignore_index=True)
    
    return block[['Period', 'Row', 'Winner', 'Non-winner', 't-test']]

# Build alternative Table 1
period_levels = ['1960–80', '1980–2000', '2000–20', 'Full sample']
table1_alt = pd.concat([make_period_block_alt(df_alt_panel, p) for p in period_levels], ignore_index=True)

print("TABLE 1 (ALTERNATIVE): Chain Linked Volume (L) + National Currency (XDC)")
print("=" * 80)
print("Note: GDP is in national currency, NOT USD PPP - only YoY growth is comparable")
print("=" * 80)
display(table1_alt)

TABLE 1 (ALTERNATIVE): Chain Linked Volume (L) + National Currency (XDC)
Note: GDP is in national currency, NOT USD PPP - only YoY growth is comparable


,Period,Row,Winner,Non-winner,t-test
0,1960–80,"GDP (millions, national currency)",0.23 (0.04),6.79 (11.67),-5.02***
1,1960–80,Year-on-Year GDP growth (%),2.64 (2.72),5.49 (3.92),-6.16***
2,1980–2000,"GDP (millions, national currency)",0.40 (0.13),17.81 (57.95),-4.68***
3,1980–2000,Year-on-Year GDP growth (%),2.48 (1.72),3.73 (3.67),-5.18***
4,2000–20,"GDP (millions, national currency)",0.49 (0.19),26.71 (88.19),-6.51***
5,2000–20,Year-on-Year GDP growth (%),0.98 (3.29),2.47 (3.86),-7.97***
6,Full sample,"GDP (millions, national currency)",0.44 (0.18),23.29 (79.36),-8.28***
7,Full sample,Year-on-Year GDP growth (%),1.76 (3.29),3.15 (4.04),-9.36***
8,Full sample,Number of countries,6,37,
9,Full sample,Number of observations,827,4462,


In [28]:
# Compare YoY growth rates: L vs LR vs Paper
print("COMPARISON: YoY GDP Growth (Full Sample)")
print("=" * 70)
print(f"{'Source':<30} {'Winner':<20} {'Non-winner':<20}")
print("-" * 70)

# Paper values
print(f"{'Paper (Table 1)':<30} {'2.33 (3.24)':<20} {'3.22 (3.78)':<20}")

# LR values (from earlier table1_manual)
lr_full = table1_manual[
    (table1_manual['Period'] == 'Full sample') & 
    (table1_manual['Row'].str.contains('Year-on-Year'))
]
if len(lr_full) > 0:
    print(f"{'LR (Chain linked rebased)':<30} {lr_full['Winner'].values[0]:<20} {lr_full['Non-winner'].values[0]:<20}")

# L values (from table1_alt)
l_full = table1_alt[
    (table1_alt['Period'] == 'Full sample') & 
    (table1_alt['Row'].str.contains('Year-on-Year'))
]
if len(l_full) > 0:
    print(f"{'L (Chain linked)':<30} {l_full['Winner'].values[0]:<20} {l_full['Non-winner'].values[0]:<20}")

COMPARISON: YoY GDP Growth (Full Sample)
Source                         Winner               Non-winner          
----------------------------------------------------------------------
Paper (Table 1)                2.33 (3.24)          3.22 (3.78)         
LR (Chain linked rebased)      2.52 (3.29)          3.39 (3.97)         
L (Chain linked)               1.76 (3.29)          3.15 (4.04)         
